# PM2.5 Annual Statistics Processing

This notebook demonstrates a complete geospatial data processing workflow:

1. **Unzip** archived global PM2.5 data
2. **Clip** monthly rasters to country boundaries
3. **Compute** annual statistics (mean, sum, max, min, median, std)
4. **Save** results to organized output directories
5. **Cleanup** temporary files

**Data Source:** V6GL02.02.CNNPM25.Global.2019.zip (monthly global PM2.5 concentrations)

**Countries:** Togo (TGO), Ghana (GHA), Tanzania (TZA), Kenya (KEN)

## Setup

In [ ]:
import sys
from pathlib import Path
import zipfile
import shutil
import rasterio
from rasterio.mask import mask
from rasterio.plot import show
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# Add src to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

from geoworkflow.utils.temporal_raster_utils import compute_temporal_average
from geoworkflow.utils.config_loader import ConfigLoader

print("✓ Imports successful")

In [ ]:
# Define paths
DATA_ROOT = project_root.parent / "data"
ZIP_FILE = DATA_ROOT / "00_source" / "archives" / "pm25" / "V6GL02.04.CNNPM25.GL.2019.archive.zip"
TEMP_DIR = DATA_ROOT / "temp" / "pm25_processing"
OUTPUT_BASE = DATA_ROOT / "emissions" / "pm25" / "2019"

# Countries to process
COUNTRIES = ["TGO", "GHA", "TZA", "KEN"]

# Load country boundaries
boundaries_path = ConfigLoader.get_africa_boundaries_path()
boundaries_columns = ConfigLoader.get_africa_boundaries_columns()

print(f"Zip file: {ZIP_FILE}")
print(f"Zip exists: {ZIP_FILE.exists()}")
print(f"Temp directory: {TEMP_DIR}")
print(f"Output base: {OUTPUT_BASE}")
print(f"Countries: {', '.join(COUNTRIES)}")

---

## Step 1: Unzip Global PM2.5 Data

Extract monthly global PM2.5 rasters to a temporary directory.

In [ ]:
# Check if zip file exists
if not ZIP_FILE.exists():
    raise FileNotFoundError(
        f"PM2.5 zip file not found: {ZIP_FILE}\n"
        f"Please ensure the file exists at this location."
    )

# Create temp directory
TEMP_DIR.mkdir(parents=True, exist_ok=True)

print(f"Unzipping {ZIP_FILE.name}...")
print(f"Destination: {TEMP_DIR}")
print()

# Unzip the file
with zipfile.ZipFile(ZIP_FILE, 'r') as zip_ref:
    # Get list of files
    file_list = zip_ref.namelist()
    tif_files = [f for f in file_list if f.endswith('.tif') or f.endswith('.TIF')]
    
    print(f"Found {len(tif_files)} TIFF files in archive:")
    for f in tif_files[:5]:
        print(f"  - {f}")
    if len(tif_files) > 5:
        print(f"  ... and {len(tif_files) - 5} more")
    print()
    
    # Extract all files
    print("Extracting...")
    zip_ref.extractall(TEMP_DIR)

print(f"✓ Extraction complete!")

# List extracted TIFF files
extracted_tifs = sorted(TEMP_DIR.rglob("*.tif"))
print(f"\nExtracted {len(extracted_tifs)} TIFF files")

---

## Step 2: Clip Monthly Rasters to Country Boundaries

For each country, clip all monthly global rasters to the country boundary.

In [ ]:
# Load country boundaries
boundaries = gpd.read_file(boundaries_path)
iso3_col = boundaries_columns['iso3']

print("Clipping rasters to country boundaries...")
print("="*70)

# Store paths to clipped files for each country
country_monthly_files = {}

for country_iso in COUNTRIES:
    print(f"\n{country_iso}:")
    
    # Get country geometry
    country_data = boundaries[boundaries[iso3_col] == country_iso]
    if len(country_data) == 0:
        print(f"  ⚠ Country {country_iso} not found in boundaries file")
        continue
    
    country_geom = country_data.iloc[0].geometry
    country_crs = boundaries.crs
    
    # Create output directory for this country
    country_dir = OUTPUT_BASE / country_iso / "monthly"
    country_dir.mkdir(parents=True, exist_ok=True)
    
    clipped_files = []
    
    # Process each monthly TIFF
    for monthly_tif in tqdm(extracted_tifs, desc=f"  Clipping {country_iso}"):
        # Open source raster
        with rasterio.open(monthly_tif) as src:
            # Reproject geometry to match raster CRS
            country_gdf = gpd.GeoDataFrame(
                {'geometry': [country_geom]},
                crs=country_crs
            )
            
            if country_gdf.crs != src.crs:
                country_gdf = country_gdf.to_crs(src.crs)
            
            # Clip raster to country boundary
            try:
                out_image, out_transform = mask(
                    src,
                    country_gdf.geometry,
                    crop=True,
                    all_touched=True
                )
                
                # Create output filename
                output_filename = f"{country_iso}_{monthly_tif.name}"
                output_path = country_dir / output_filename
                
                # Write clipped raster
                out_meta = src.meta.copy()
                out_meta.update({
                    "driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform,
                    "compress": "lzw"
                })
                
                with rasterio.open(output_path, "w", **out_meta) as dest:
                    dest.write(out_image)
                
                clipped_files.append(output_path)
                
            except Exception as e:
                print(f"  ⚠ Error clipping {monthly_tif.name}: {e}")
    
    country_monthly_files[country_iso] = clipped_files
    print(f"  ✓ Created {len(clipped_files)} clipped monthly files")

print("\n" + "="*70)
print("✓ Clipping complete for all countries")

---

## Step 3: Compute Annual Statistics

Use the temporal_raster_utils to compute multiple annual statistics:
- **Mean**: Average PM2.5 concentration
- **Sum**: Total PM2.5 (cumulative)
- **Max**: Maximum monthly concentration
- **Min**: Minimum monthly concentration
- **Median**: Median concentration
- **Std**: Standard deviation

In [ ]:
# Statistics to compute
STATISTICS = ['mean', 'sum', 'max', 'min', 'median', 'std']

print("Computing annual statistics for each country...")
print("="*70)

processing_results = {}

for country_iso in COUNTRIES:
    if country_iso not in country_monthly_files:
        print(f"\n{country_iso}: Skipped (no monthly files)")
        continue
    
    print(f"\n{country_iso}:")
    print("-" * 70)
    
    monthly_dir = OUTPUT_BASE / country_iso / "monthly"
    annual_dir = OUTPUT_BASE / country_iso / "annual"
    annual_dir.mkdir(exist_ok=True)
    
    country_results = {}
    
    for statistic in STATISTICS:
        try:
            result = compute_temporal_average(
                input_dir=monthly_dir,
                output_path=annual_dir / f"{country_iso}_pm25_2019_annual_{statistic}.tif",
                statistic=statistic,
                require_complete=False,  # Allow partial data
                verbose=False,
                overwrite=True
            )
            
            country_results[statistic] = result
            print(f"  ✓ {statistic:8s}: {result['input_count']:2d} months processed")
            
        except Exception as e:
            print(f"  ✗ {statistic:8s}: Error - {str(e)[:60]}")
    
    processing_results[country_iso] = country_results

print("\n" + "="*70)
print("✓ Annual statistics computed for all countries")

---

## Step 4: Visualize Results

Compare annual mean PM2.5 across countries.

In [ ]:
# Determine grid size based on number of countries
n_countries = len([c for c in COUNTRIES if c in processing_results])
n_cols = 2
n_rows = (n_countries + 1) // 2

fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 6*n_rows))
if n_rows == 1:
    axes = axes.reshape(1, -1)

idx = 0
for country_iso in COUNTRIES:
    if country_iso not in processing_results:
        continue
    
    if 'mean' not in processing_results[country_iso]:
        continue
    
    result = processing_results[country_iso]['mean']
    
    row = idx // n_cols
    col = idx % n_cols
    ax = axes[row, col]
    
    with rasterio.open(result['output_file']) as src:
        data = src.read(1)
        data_masked = np.ma.masked_equal(data, src.nodata)
        
        im = ax.imshow(data_masked, cmap='RdYlGn_r', interpolation='nearest')
        ax.set_title(
            f"{country_iso}: Annual Mean PM2.5 (2019)\n"
            f"Mean: {data_masked.mean():.1f} µg/m³",
            fontsize=12, fontweight='bold'
        )
        ax.axis('off')
        plt.colorbar(im, ax=ax, label='PM2.5 (µg/m³)', shrink=0.7)
    
    idx += 1

# Hide unused subplots
for i in range(idx, n_rows * n_cols):
    row = i // n_cols
    col = i % n_cols
    axes[row, col].axis('off')

plt.suptitle('PM2.5 Annual Mean Comparison (2019)', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

### Compare All Statistics for One Country

In [ ]:
# Choose first available country
example_country = COUNTRIES[0] if COUNTRIES[0] in processing_results else None

if example_country and processing_results[example_country]:
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    axes = axes.flatten()
    
    for idx, stat in enumerate(STATISTICS):
        if stat in processing_results[example_country]:
            result = processing_results[example_country][stat]
            
            with rasterio.open(result['output_file']) as src:
                data = src.read(1)
                data_masked = np.ma.masked_equal(data, src.nodata)
                
                im = axes[idx].imshow(data_masked, cmap='RdYlGn_r', interpolation='nearest')
                axes[idx].set_title(
                    f"{stat.upper()}\n"
                    f"Range: {data_masked.min():.1f} - {data_masked.max():.1f} µg/m³",
                    fontsize=11, fontweight='bold'
                )
                axes[idx].axis('off')
                plt.colorbar(im, ax=axes[idx], shrink=0.7)
    
    plt.suptitle(f'{example_country}: PM2.5 Annual Statistics Comparison (2019)',
                 fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()
else:
    print("No results available for visualization")

---

## Step 5: Summary Statistics Table

In [ ]:
# Create summary table
summary_data = []

for country_iso in COUNTRIES:
    if country_iso not in processing_results:
        continue
    
    country_stats = {}
    country_stats['Country'] = country_iso
    
    for stat in STATISTICS:
        if stat in processing_results[country_iso]:
            result = processing_results[country_iso][stat]
            
            with rasterio.open(result['output_file']) as src:
                data = src.read(1)
                data_masked = np.ma.masked_equal(data, src.nodata)
                country_stats[stat.capitalize()] = f"{data_masked.mean():.2f}"
        else:
            country_stats[stat.capitalize()] = "N/A"
    
    country_stats['Files'] = result['input_count'] if result else 0
    summary_data.append(country_stats)

df_summary = pd.DataFrame(summary_data)

print("\nPM2.5 Annual Statistics Summary (2019)")
print("Values shown: spatial mean of each statistic (µg/m³)")
print("="*100)
print(df_summary.to_string(index=False))

# Save summary to CSV
summary_path = OUTPUT_BASE / "pm25_2019_summary_statistics.csv"
df_summary.to_csv(summary_path, index=False)
print(f"\n✓ Summary saved to: {summary_path}")

---

## Step 6: Cleanup Temporary Files

Delete the unzipped global rasters to save disk space.

In [ ]:
print("Cleaning up temporary files...")
print(f"Removing: {TEMP_DIR}")

# Calculate size before deletion
temp_size = sum(f.stat().st_size for f in TEMP_DIR.rglob('*') if f.is_file()) / (1024**2)
print(f"Temp directory size: {temp_size:.1f} MB")

# Remove temporary directory
if TEMP_DIR.exists():
    shutil.rmtree(TEMP_DIR)
    print(f"✓ Temporary directory removed")
else:
    print(f"⚠ Temporary directory not found")

print("\n✓ Cleanup complete!")

---

## Step 7: Processing Summary

In [ ]:
print("="*70)
print("PROCESSING COMPLETE")
print("="*70)
print()

for country_iso in COUNTRIES:
    if country_iso not in processing_results:
        print(f"{country_iso}: Not processed")
        continue
    
    country_dir = OUTPUT_BASE / country_iso
    monthly_files = list((country_dir / "monthly").glob("*.tif")) if (country_dir / "monthly").exists() else []
    annual_files = list((country_dir / "annual").glob("*.tif")) if (country_dir / "annual").exists() else []
    
    total_size = sum(f.stat().st_size for f in monthly_files + annual_files) / (1024**2)
    
    print(f"{country_iso}:")
    print(f"  Monthly files: {len(monthly_files)}")
    print(f"  Annual statistics: {len(annual_files)}")
    print(f"  Total size: {total_size:.1f} MB")
    print(f"  Location: {country_dir}")
    print()

print("="*70)
print("Output structure:")
print(f"{OUTPUT_BASE}/")
print("  ├── TGO/")
print("  │   ├── monthly/    (12 clipped monthly TIFFs)")
print("  │   └── annual/     (6 annual statistics TIFFs)")
print("  ├── GHA/")
print("  ├── TZA/")
print("  ├── KEN/")
print("  └── pm25_2019_summary_statistics.csv")

---

## Optional: Delete Monthly Files (Keep Only Annual Statistics)

If you want to save disk space and only keep the annual statistics:

In [ ]:
# OPTIONAL: Uncomment to delete monthly files after computing statistics

# delete_monthly = False  # Set to True to delete monthly files

# if delete_monthly:
#     print("Deleting monthly files (keeping only annual statistics)...")
#     
#     for country_iso in COUNTRIES:
#         monthly_dir = OUTPUT_BASE / country_iso / "monthly"
#         if monthly_dir.exists():
#             monthly_size = sum(f.stat().st_size for f in monthly_dir.glob("*.tif")) / (1024**2)
#             shutil.rmtree(monthly_dir)
#             print(f"  ✓ {country_iso}: Removed {monthly_size:.1f} MB of monthly files")
#     
#     print("\n✓ Monthly files deleted")
# else:
#     print("Monthly files retained (set delete_monthly=True to remove)")

print("Monthly files retained (uncomment code above to delete)")